### Description:

- Reads sun glare dataset and visualizes it
- Also overlaps other metrics

In [25]:
import folium
import math

def convert_anti_clockwise_east_heading_to_clockwise_north(anti_clockwise_heading):
    # convert the heading from anti-clockwise east to clockwise north
    clockwise_heading = 90 - anti_clockwise_heading
    if clockwise_heading < 0:
        clockwise_heading = 360 + clockwise_heading
    return clockwise_heading


def get_circle_color_for_sun_glare(sun_glare_row):

    if sun_glare_row['angle_risk'] == True:
        if sun_glare_row['has_sun_glare'] == True:
            # sun glare not blocked
            return 'red'
        else:
            # sun glare blocked
            return 'yellow'
    # no chance of sun glare
    return 'green'



def visualize_just_data(sun_glare_dataset, segments_dataset, center_point):

    m = folium.Map(location=center_point, zoom_start=16)

    # TODO find a way to draw lines between the points that are going in the same heading
    # ex: 2 way streets will have 2 lines, one for each direction

    # TODO maybe request lineString from the osnmx edges, then use that to draw the lines (instead of straight lines) 

    #        Yellow       Red
    #    ----node---- ----node----   -->
    #        Green        Green
    #    ----node---- ----node----   <--
    # half the ditance between the two nodes is colored one color, the other half is colored another color

    row_counter = 0
    for _, row in sun_glare_dataset.iterrows():

        
        
        row_counter += 1
        circle_color = get_circle_color_for_sun_glare(row)

        lat = row['lat']
        long = row['long']
        heading = convert_anti_clockwise_east_heading_to_clockwise_north(row['heading'])

        # folium.CircleMarker(
        #     location=[lat, long],
        #     radius=5,
        #     color=circle_color,
        #     fill=True,
        #     fill_opacity=0.6,
        #     popup=f"(lat: {lat}, long: {long}, heading: {heading})"

        # ).add_to(m)

        distance = 0.00025  # Adjust distance for visual clarity on the map
        adjusted_heading = heading  
        heading_rad = math.radians(adjusted_heading)

        end_lat = lat + distance * math.cos(heading_rad)  # Latitude change uses cos
        end_long = long + distance * math.sin(heading_rad)  # Longitude change uses sin
        
        # Buckets for different heading ranges
        offset_buckets = {
            "north": (0.000005, 0.00001),  # x_offset, y_offset
            "east": (0.00001, 0.000005),
            "south": (-0.000005, -0.00001),
            "west": (-0.00001, -0.000005),
        }

        # Determine the bucket based on heading
        if 0 <= heading < 90:
            x_offset, y_offset = offset_buckets["north"]
        elif 90 <= heading < 180:
            x_offset, y_offset = offset_buckets["east"]
        elif 180 <= heading < 270:
            x_offset, y_offset = offset_buckets["south"]
        elif 270 <= heading < 360:
            x_offset, y_offset = offset_buckets["west"]
        else:
            x_offset, y_offset = 0, 0  # Default offset for invalid heading

        # Apply the offsets
        lat += y_offset
        long += x_offset
        end_lat += y_offset
        end_long += x_offset



        # Draw a line indicating the direction of the heading
        folium.PolyLine(
            locations=[(lat, long), (end_lat, end_long)],
            color=circle_color,
            weight=6,
            opacity=0.7
        ).add_to(m)

    m.save(f"saved_sun_glare_map.html")
    print(f"Map created with {row_counter} detections.")




In [26]:
import pandas as pd

def main():
    sun_glare_dataset_path = "data/sun_glare_dict.csv"
    segments_dataset_path = "data/dc_segments.csv"

    sun_glare_data = pd.read_csv(sun_glare_dataset_path)
    segments_data = pd.read_csv(segments_dataset_path)

    center_point = (38.89565719232077, -77.04168192501736)

    visualize_just_data(sun_glare_data, segments_data, center_point)


main()

Map created with 163 detections.
